# Port of Vancouver Data

- Objective: forecast future month TEU of Port of Vancouver, both import and export.
- Data Source: [container statistics monthly 2008 - 2024](https://www.portvancouver.com/media/documents/container-statistics-monthly-2008-2024)
- Approach: Meta Prophet
- Result: Cleaned monthly and yearly time series data in .csv format

### Required Packages

In [801]:
from tabula import read_pdf
import pandas as pd
import itertools
import numpy as np
import copy

from prophet import Prophet
from prophet.plot import add_changepoints_to_plot
from prophet.diagnostics import cross_validation, performance_metrics, prophet_copy
from prophet.plot import plot_cross_validation_metric
from prophet.serialize import model_to_json, model_from_json


from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

## Process Data from .pdf

In [257]:
file_name = 'container-statistics-monthly-2008-2024.pdf'
path_name = './data/port_of_vancouver'

#reads table from pdf file
df = read_pdf(path_name + '/' + file_name, output_format='dataframe',pages='all')
df = df[0]
df
# df = read_pdf("https://www.portvancouver.com/media/documents/container-statistics-monthly-2008-2024", output_format='json')

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,TEU COUNT,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,CONTAINER COUNT,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,YEAR/MONTH,NaN,TOTAL,NaN,NaN,IMPORT,NaN,NaN,EXPORT,NaN,NaN,TOTAL,NaN,NaN,IMPORT,NaN,NaN,EXPORT,NaN
1,NaN,TOTAL,EMPTY,LADEN,TOTAL,EMPTY,LADEN,TOTAL,EMPTY,LADEN,TOTAL,EMPTY,LADEN,TOTAL,EMPTY,LADEN,TOTAL,EMPTY,LADEN
2,2008,"2,492,107","338,291","2,153,816","1,294,308","55,958","1,238,350","1,197,799","282,334","915,465","1,435,127","199,852","1,235,275","744,430","30,588","713,842","690,697","169,264","521,433"
3,January,"185,511","17,684","167,827","101,683","4,087","97,597","83,828","13,597","70,230","108,350","10,863","97,487","59,001","2,373","56,628","49,349","8,490","40,859"
4,February,"211,837","19,606","192,231","108,924","3,259","105,664","102,914","16,347","86,567","122,113","11,703","110,410","62,630","1,827","60,803","59,483","9,876","49,607"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,July,"307,750","77,841","229,909","169,265",101,"169,164","138,485","77,740","60,745","170,513","41,889","128,624","94,458",76,"94,382","76,055","41,813","34,242"
233,August,"303,361","84,860","218,501","155,706",30,"155,676","147,655","84,830","62,825","166,850","45,732","121,118","85,646",12,"85,634","81,204","45,720","35,484"
234,September,"303,589","81,401","222,188","153,249",627,"152,622","150,340","80,774","69,566","167,859","43,683","124,176","84,733",602,"84,131","83,126","43,081","40,045"
235,October,"282,348","70,018","212,331","150,373","1,530","148,843","131,976","68,488","63,488","156,474","36,556","119,918","83,807","1,515","82,292","72,667","35,041","37,626"


In [258]:
# column names
colname = ['TEU COUNT']*3*3 + ['Container COUNT']*3*3
colname_2 = ['Total']*3 + ['Import']*3 + ['Export']*3
colname_3 = ['Total', 'Empty', 'Laden']
colname_combined = ['Year/Month'] + [colname[i] + '_' + colname_2[i % len(colname_2)] + '_' + colname_3[i % len(colname_3)] for i in range(len(colname))]
df.columns = colname_combined
drop_rows = [i for i in df.index if df['Year/Month'][i] == 'YEAR/MONTH' or pd.isna(df['Year/Month'][i])]
df = df.drop(drop_rows, axis=0, inplace=False)

### Data Quality

In [259]:
# change data type
try:
    df.replace(',','', regex=True, inplace=True)
    df = df.astype({col: int for col in colname_combined[1:]})
except Exception as error:
    print('ERROR: ', error)
# cannot change datatype to integer

ERROR:  invalid literal for int() with base 10: '#REF!': Error while type casting for column 'TEU COUNT_Export_Laden'


In [260]:
error_row = df[df['TEU COUNT_Export_Laden'] == '#REF!']
error_row_idx = error_row.index[0]
error_row

,Year/Month,TEU COUNT_Total_Total,TEU COUNT_Total_Empty,TEU COUNT_Total_Laden,TEU COUNT_Import_Total,TEU COUNT_Import_Empty,TEU COUNT_Import_Laden,TEU COUNT_Export_Total,TEU COUNT_Export_Empty,TEU COUNT_Export_Laden,Container COUNT_Total_Total,Container COUNT_Total_Empty,Container COUNT_Total_Laden,Container COUNT_Import_Total,Container COUNT_Import_Empty,Container COUNT_Import_Laden,Container COUNT_Export_Total,Container COUNT_Export_Empty,Container COUNT_Export_Laden
183,2021,3678952,891509,2787443,1923621,14604,1909017,876905,878426,#REF!,2061176,483201,1577975,1080625,10689 1069936,NaN,980551,472512,508039


In [261]:
# deal with #REF!
row_end_idx = df[df['Year/Month'] == '2022'].index[0]
error_columns = ['TEU COUNT_Export_' + x for x in ['Total', 'Empty', 'Laden']]
tmp_df = df.loc[[i for i in range(error_row_idx, row_end_idx)], ['Year/Month'] + error_columns]
tmp_df.reset_index(drop=True, inplace=True)
correct_value = tmp_df.loc[1:, ].astype({col: int for col in error_columns}).drop(['Year/Month'], axis=1).sum(axis=0)
tmp_df.loc[tmp_df.shape[0]] = ['Correct Total'] + list(correct_value)
tmp_df.loc[[0, tmp_df.shape[0]-1]]

,Year/Month,TEU COUNT_Export_Total,TEU COUNT_Export_Empty,TEU COUNT_Export_Laden
0,2021,876905,878426,#REF!
13,Correct Total,1755333,876907,878429


In [262]:
# replace with correct data
df.loc[error_row_idx, error_columns] = correct_value
# double check result
df.loc[error_row_idx]

Year/Month                               2021
TEU COUNT_Total_Total                 3678952
TEU COUNT_Total_Empty                  891509
TEU COUNT_Total_Laden                 2787443
TEU COUNT_Import_Total                1923621
TEU COUNT_Import_Empty                  14604
TEU COUNT_Import_Laden                1909017
TEU COUNT_Export_Total                1755333
TEU COUNT_Export_Empty                 876907
TEU COUNT_Export_Laden                 878429
Container COUNT_Total_Total           2061176
Container COUNT_Total_Empty            483201
Container COUNT_Total_Laden           1577975
Container COUNT_Import_Total          1080625
Container COUNT_Import_Empty    10689 1069936
Container COUNT_Import_Laden              NaN
Container COUNT_Export_Total           980551
Container COUNT_Export_Empty           472512
Container COUNT_Export_Laden           508039
Name: 183, dtype: object

In [263]:
# deal with NA and wrong alignment
na_colname = df.columns[df.isna().any()][0]
print(na_colname)
na_columns = ['Container COUNT_Import_' + x for x in ['Total', 'Empty', 'Laden']]
na_rows = [i for i in df.index if pd.isna(df.loc[i, na_colname])]
df.loc[na_rows, na_columns].head()

Container COUNT_Import_Laden


,Container COUNT_Import_Total,Container COUNT_Import_Empty,Container COUNT_Import_Laden
170,1056610,48199 1008411,NaN
171,84669,3257 81412,NaN
172,69532,4841 64691,NaN
173,69094,5051 64043,NaN
174,90636,5279 85357,NaN


In [264]:
split_list = df.loc[na_rows, 'Container COUNT_Import_Empty'].str.split(' ').to_list()
empty_val = [i[0] for i in split_list]
laden_val = [i[1] for i in split_list]
# replace with correct data
df.loc[na_rows, na_columns[1]] = empty_val
df.loc[na_rows, na_columns[2]] = laden_val
# double check the replacement result
df.loc[na_rows, na_columns].head()

,Container COUNT_Import_Total,Container COUNT_Import_Empty,Container COUNT_Import_Laden
170,1056610,48199,1008411
171,84669,3257,81412
172,69532,4841,64691
173,69094,5051,64043
174,90636,5279,85357


### Data Type

In [265]:
# change data type
try:
    df.replace(',','', regex=True, inplace=True)
    df = df.astype({col: int for col in colname_combined[1:]})
except Exception as error:
    print('ERROR: ', error)
# cannot change datatype to integer

### Split Yearly and Monthly Data

In [266]:
years = [str(x) for x in range(2008,2025)]
year_index = [i for i in df.index if df['Year/Month'][i] in years]
total_year_series.reset_index(drop=True,inplace=True)
total_year_series = copy.deepcopy(df.loc[year_index])
total_year_series.rename(columns={"Year/Month": "year"}, inplace=True)
total_year_series['ds'] = pd.to_datetime(total_year_series['year'], format = '%Y') 

month_series = copy.deepcopy(df.drop(year_index, axis=0, inplace=False))
month_series.reset_index(drop=True,inplace=True)
month_series.rename(columns={"Year/Month": "month"}, inplace=True)
month_series['year'] = [year for year in years for i in range(12)][:month_series.shape[0]]
month_series['ds'] = pd.to_datetime(month_series['year'] + ' ' + month_series['month'], format='%Y %B')

### Save Data

In [829]:
total_year_series.to_csv(path_name + '/' + file_name.replace('.pdf', '.csv').replace('monthly', 'yearly'), sep=',', index=True, header=True)

month_series.to_csv(path_name + '/' + file_name.replace('.pdf', '.csv'), sep=',', index=True, header=True)